В данном ноутбуке вам предстоит реализовать алгоритм Виолы-Джонса для детекции лиц.

Полное описание алгоритма находится в статье https://www.cs.cmu.edu/~efros/courses/LBMV07/Papers/viola-cvpr-01.pdf

Алгоритм является довольно ресурсоемким, поэтому если будет не хватать памяти, то попробуйте уменьшить количество изображений в наборе. На тысяче изображений обоих классов все еще можно получить неплохие результаты.

# Считаем данные



In [70]:
import numpy as np
import os

from tqdm.notebook import tqdm

from skimage import io
from skimage.transform import resize
import abc

import numba

In [71]:
def get_all_images(starting_dir, nsamples=None):
    images = []
    stop = False
    extensions = ["pgm", "jpeg", "jpg", "png"]
    pbar = tqdm(os.walk(starting_dir), desc=f'loading from {starting_dir}, {len(images)} found')
    for dir_name, _, filenames in pbar:
        for filename in filenames:
            extension = os.path.splitext(filename)[1][1:]
            if extension in extensions:
                image = io.imread(os.path.join(dir_name, filename))
                images.append(image)
                pbar.set_description(f'loading from {starting_dir}, {len(images)} found')
                stop = nsamples and len(images) >= nsamples
            if stop:
                break
        if stop:
            pbar.set_description(f'{starting_dir} done, {len(images)} found')
            break

    return images

In [72]:
positives = get_all_images('data/positives')
n_positives = len(positives)
negatives = get_all_images('data/negatives')

loading from data/positives, 0 found: 0it [00:00, ?it/s]

loading from data/negatives, 0 found: 0it [00:00, ?it/s]

## Зафиксируем размер окна, на котором будет работать классификатор

In [73]:
image_canonical_size = 24

# Препроцессинг

In [74]:
# Вычтем из изображения среднее и поделим на стандартное отклонение
def normalize_images(images):
    result = []
    for image in tqdm(images, desc='normalizing'):
        mean, std = image.mean(), image.std()
        result.append((image - mean) / std)
    return result

## Препроцессинг изображений с лицами

* Нормируем яркость, чтобы не учитывать освещенность
* Преобразуем к 24 * 24

In [75]:
def prepare_positives(images, size):
    result = []
    normilized_images = normalize_images(images)
    for normilized_image in tqdm(normilized_images, desc='resizing'):
        resized_image = resize(normilized_image, (size, size), mode='constant').astype(np.float32)
        result.append(resized_image)
    return result

In [76]:
positives_prepared = prepare_positives(positives, image_canonical_size)

normalizing:   0%|          | 0/3206 [00:00<?, ?it/s]

resizing:   0%|          | 0/3206 [00:00<?, ?it/s]

## Препроцессинг изображений без лиц

* Вырежем случайные квадраты из негативных изображений
* Нормируем яркость
* Преобразуем к 24 * 24

In [77]:
from random import randint


def prepare_negatives(images, sample_size, size):
    norm_images = normalize_images(images)
    crops = []
    for _ in tqdm(range(0, sample_size), desc='cropping'):
        image_ind = randint(0, len(norm_images) - 1)
        image = norm_images[image_ind]
        w, h = image.shape
        max_r = min(w, h)
        r = randint(size, max_r)
        x, y = randint(0, w - max_r), randint(0, h - max_r)
        crop = image[x: x + r, y: y + r]
        crop = resize(crop, (size, size), mode='constant').astype(np.float32)
        crops.append(crop)
    return crops

In [78]:
# Возьмем столько же негативных изображений, сколько позитивных
n_negatives = len(negatives)
negatives_prepared = prepare_negatives(negatives, n_negatives, image_canonical_size)

normalizing:   0%|          | 0/3019 [00:00<?, ?it/s]

cropping:   0%|          | 0/3019 [00:00<?, ?it/s]

## Проверим, что данные имеют правильный формат

In [79]:
def image_has_correct_format(image, shape=(image_canonical_size, image_canonical_size)):
    return image.shape == shape


assert (len(positives_prepared) == n_positives)
assert (all(
    [image_has_correct_format(im) for im in positives_prepared]
))

assert (len(negatives_prepared) == n_negatives)
assert (all(
    [image_has_correct_format(im) for im in negatives_prepared]
))

## Интегральное изображение

Чтобы эффективно вычислять признаки Хаара нам понадобится специальное представление изображения под названием Интегральное изображение.

Интегральное изображение I -- это матрица, совпадающая по размерам с исходным изображением, в которой I(x, y) = сумме всех интенсивностей пикселей левее и выше (x, y) (включая [x, y]). Такое представление позволяет очень быстро находить сумму интенсивностей пикселей в любом подпрямоугольнике изображения (что нужно для вычисления признаков Хаара).

Принцип работы проще всего понять графически:

<img src="img/integral.png",width=300,height=300>

Чтобы найти площадь серого прямоугольника мы можем использовать только прямоугольники, начинающиеся из (0,0). Именно эти суммы мы и храним в интегральном изображении.

Несложно видеть, что 

S = C - B - D + A

Вам необходимо реализовать интегральное изображение самостоятельно

In [80]:
class IntegralImage:
    def __init__(self, image):
        # hint: воспользуйтесь numpy.cumsum два раза, чтобы получить двумерную кумулятивную сумму
        self.integral_image = np.cumsum(np.cumsum(image, axis=1), axis=0)


    def sum(self, x1, y1, x2, y2):
        '''
        Сумма подмассива
        
        На входе:
            x1, y1 -- координаты левого нижнего угла прямоугольника запроса
            x2, y2 -- координаты верхнего правого угла прямоугольника запроса
            
        На выходе:
            Сумма подмассива [x1..x2, y1..y2]
        '''
        return sum_IntegralImage(self.integral_image, x1, y1, x2, y2)
        # assert (x1 <= x2)
        # assert (y1 <= y2)
        # b1 = self.integral_image[x1 - 1, y1 - 1] if x1 and y1 else 0
        # b2 = self.integral_image[x2, y1 - 1] if y1 else 0
        # b3 = self.integral_image[x1 - 1, y2] if x1 else 0
        # b4 = self.integral_image[x2, y2]
        # return b4 - b2 - b3 + b1

@numba.jit()
def sum_IntegralImage(integral_image, x1, y1, x2, y2):
    assert (x1 <= x2)
    assert (y1 <= y2)
    b1 = integral_image[x1 - 1, y1 - 1] if x1 and y1 else 0
    b2 = integral_image[x2, y1 - 1] if y1 else 0
    b3 = integral_image[x1 - 1, y2] if x1 else 0
    b4 = integral_image[x2, y2]
    return b4 - b2 - b3 + b1

## Протестируем

In [81]:
test_image = np.asarray([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

test_integral_image = IntegralImage(test_image)
assert (test_integral_image.sum(0, 0, 2, 2) == sum(range(0, 10)))
assert (test_integral_image.sum(0, 0, 0, 0) == 1)
assert (test_integral_image.sum(0, 0, 2, 0) == 12)
assert (test_integral_image.sum(0, 1, 1, 2) == 16)

## Подготовим интегральные изображения

In [82]:
integral_positives = [IntegralImage(im) for im in positives_prepared]
integral_negatives = [IntegralImage(im) for im in negatives_prepared]

## Признаки Хаара

В оригинальной работе использовались признаки следующего вида:

<img src="haar_features_1.jpg">

Значение признака = s1 - s2, где s1 = сумма интенсивностей в светлой области, s2 = сумма интенсивностей в темной области
Не обязательно использовать все эти признаки, но рекомендуется попробовать. Ниже необходимо реализовать их.

Признак задается координатами левого верхнего угла, шириной и высотой окна. Считаем, что ось x идет сверху вниз, а ось y -- справа-налево. w -- это размер окна по оси x, h -- по оси y.

In [83]:
# Общий интерфейс для всех классов признаков

class HaarFeature(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def compute_value(self, integral_image):
        '''
        Функция, вычисляющая и возвращающая значение признака

        На входе:
            integral_image -- IntegralImage

        На выходе:
            Значение признака
        '''
        pass

    def __repr__(self):
        return "Feature {}, {}, {}, {}".format(self.x, self.y, self.w, self.h)

### Пример реализации признака (a)

In [84]:
class HaarFeatureVerticalTwoSegments(HaarFeature):
    def __init__(self, x, y, w, h):
        assert (h % 2 == 0)
        assert (x >= 0)
        assert (y >= 0)
        assert (w >= 2)
        assert (h >= 2)
        self.x = x
        self.y = y
        self.w = w
        self.h = h

    def compute_value(self, integral_image):
        s1 = integral_image.sum(self.x, self.y, self.x + self.w - 1, self.y + self.h // 2 - 1)
        s2 = integral_image.sum(self.x, self.y + self.h // 2, self.x + self.w - 1, self.y + self.h - 1)
        return s1 - s2

In [85]:
test_image = np.asarray([
    [1, 1, 2, 2],
    [1, 1, 2, 2],
    [1, 1, 2, 2]
])

test_feature = HaarFeatureVerticalTwoSegments(0, 0, 3, 4)
assert (test_feature.compute_value(IntegralImage(test_image)) == 6 * 1 - 6 * 2)

По аналогии реализуйте остальные признаки

In [86]:
# признак (b)
class HaarFeatureHorizontalTwoSegments(HaarFeature):

    def __init__(self, x, y, w, h):
        assert (w % 2 == 0)
        assert (x >= 0)
        assert (y >= 0)
        assert (w >= 2)
        assert (h >= 2)
        self.x = x
        self.y = y
        self.w = w
        self.h = h

    def compute_value(self, integral_image):
        s1 = integral_image.sum(self.x, self.y, self.x + self.w // 2 - 1, self.y + self.h - 1)
        s2 = integral_image.sum(self.x + self.w // 2, self.y, self.x + self.w - 1, self.y + self.h - 1)
        return s2 - s1

In [87]:
test_image = np.asarray([
    [1, 1, 1, 1],
    [1, 1, 1, 1],
    [2, 2, 2, 2],
    [2, 2, 2, 2]
])

test_feature = HaarFeatureHorizontalTwoSegments(0, 0, 4, 4)
assert (test_feature.compute_value(IntegralImage(test_image)) == 8 * 2 - 8 * 1)

In [88]:
# признак (c)
class HaarFeatureVerticalThreeSegments(HaarFeature):

    def __init__(self, x, y, w, h):
        assert (h % 3 == 0)
        assert (x >= 0)
        assert (y >= 0)
        assert (w >= 2)
        assert (h >= 3)
        self.x = x
        self.y = y
        self.w = w
        self.h = h

    def compute_value(self, integral_image):
        s1 = integral_image.sum(self.x, self.y, self.x + self.w - 1, self.y + self.h - 1)
        s2 = integral_image.sum(self.x, self.y + self.h // 3, self.x + self.w - 1, self.y + self.h // 3 * 2 - 1)
        # print(s1, s2)
        return s1 - 2 * s2

In [89]:
test_image = np.asarray([
    [1, 1, 3, 3, 1, 1],
    [1, 1, 3, 3, 1, 1],
    [1, 1, 3, 3, 1, 1]
])

test_feature = HaarFeatureVerticalThreeSegments(0, 0, 3, 6)
print(test_feature.compute_value(IntegralImage(test_image)), 6 * 2 - 6 * 3)
assert (test_feature.compute_value(IntegralImage(test_image)) == 6 * 2 - 6 * 3)

-6 -6


In [90]:
# признак (d)
class HaarFeatureHorizontalThreeSegments(HaarFeature):
    def __init__(self, x, y, w, h):
        assert (w % 3 == 0)
        assert (x >= 0)
        assert (y >= 0)
        assert (w >= 3)
        assert (h >= 2)
        self.x = x
        self.y = y
        self.w = w
        self.h = h

    def compute_value(self, integral_image):
        s1 = integral_image.sum(self.x, self.y, self.x + self.w - 1, self.y + self.h - 1)
        s2 = integral_image.sum(self.x + self.w // 3, self.y, self.x + self.w // 3 * 2 - 1, self.y + self.h - 1)
        # print(s1, s2, self.x + self.w // 3, self.x + self.w // 3 * 2)
        return s1 - 2 * s2

In [91]:
test_image = np.asarray([
    [1, 1, 1],
    [1, 1, 1],
    [3, 3, 3],
    [3, 3, 3],
    [1, 1, 1],
    [1, 1, 1],
])

test_feature = HaarFeatureHorizontalThreeSegments(0, 0, 6, 3)
print(test_feature.compute_value(IntegralImage(test_image)), 6 * 2 - 6 * 3)
assert (test_feature.compute_value(IntegralImage(test_image)) == 6 * 2 - 6 * 3)

-6 -6


In [92]:
# признак (e)
class HaarFeatureFourSegments(HaarFeature):
    def __init__(self, x, y, w, h):
        assert (w % 2 == 0)
        assert (h % 2 == 0)
        assert (x >= 0)
        assert (y >= 0)
        assert (w >= 2)
        assert (h >= 2)
        self.x = x
        self.y = y
        self.w = w
        self.h = h

    def compute_value(self, integral_image):
        s1 = integral_image.sum(self.x, self.y, self.x + self.w // 2 - 1, self.y + self.h // 2 - 1)
        s1 += integral_image.sum(self.x + self.w // 2, self.y + self.h // 2, self.x + self.w - 1, self.y + self.h - 1)
        s2 = integral_image.sum(self.x + self.w // 2, self.y, self.x + self.w - 1, self.y + self.h // 2 - 1)
        s2 += integral_image.sum(self.x, self.y + self.h // 2, self.x + self.w // 2 - 1, self.y + self.h - 1)
        # print(s1, s2)
        return s1 - s2

In [93]:
test_image = np.asarray([
    [1, 1, 0, 0],
    [1, 1, 1, 1],
    [0, 0, 1, 1],
    [0, 0, 1, 1]
])

test_feature = HaarFeatureFourSegments(0, 0, 4, 4)
print(test_feature.compute_value(IntegralImage(test_image)), 4 * 2 - 2)
assert (test_feature.compute_value(IntegralImage(test_image)) == 4 * 2 - 2)

6 6


## Сохраним все возможные признаки

Теперь необходимо сгенерировать все возможные признаки. Для изображение 24 * 24 их получится очень много (около 150000). Чтобы сократить время вычислений, рекомендуется брать не все признаки, а некоторое подмножество. Например, можно рассматривать окна с некоторым шагом по x, y, т.к. соседние признаки все равно сильно скоррелированы. Также можно рассматривать окна не всех размеров, а тоже выбирать их с некоторым шагом. В зависимости вот возможностей компьютера, выберите шаги. 10-50 тысяч признаков должно хватить для классификатора разумной точности.

In [94]:
features_to_use = [HaarFeatureVerticalTwoSegments,
                   HaarFeatureVerticalThreeSegments,
                   HaarFeatureHorizontalTwoSegments,
                   HaarFeatureHorizontalThreeSegments,
                   HaarFeatureFourSegments]
# шаги по x,y,w,h
x_stride = 2
y_stride = 2
w_stride = 2
h_stride = 2

all_features = []
for x in range(0, image_canonical_size, x_stride):
    for y in range(0, image_canonical_size, y_stride):
        for w in range(2, image_canonical_size - x + 1, w_stride):
            for h in range(2, image_canonical_size - y + 1, h_stride):
                for feature_type in features_to_use:
                    try:
                        feature = feature_type(x, y, w, h)
                        all_features.append(feature)
                    except:
                        continue
print(f"Всего признаков: {len(all_features)}")

Всего признаков: 21684


## Вычислим все признаки на всех изображениях

In [95]:
def compute_features_for_image(integral_image, features):
    result = np.zeros(len(features))
    for ind, feature in enumerate(features):
        result[ind] = feature.compute_value(integral_image)
    return result


def compute_features(integral_images, features):
    result = np.zeros((len(integral_images), len(features)))
    for ind, integral_image in tqdm(enumerate(integral_images)):
        result[ind] = compute_features_for_image(integral_image, features)
    return result

In [96]:
positive_features = compute_features(integral_positives, all_features)
negative_features = compute_features(integral_negatives, all_features)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

## Сохраним прогресс

In [98]:
import _pickle as pickle

positive = open('positive_data.pkl', 'wb')
pickle.dump(positive_features, positive, 2)

negative = open('negative_data.pkl', 'wb')
pickle.dump(negative_features, negative, 2)